In [33]:
import json

from openai_service import get_chat_completion
from prompts import *
import pandas as pd
import json
import os
from tqdm.auto import tqdm
tqdm.pandas()

# ARC

In [23]:
arc_df = pd.read_csv('datasets/arc_challenge_train_with_prompt.csv')
arc_df.head()

,id,question,choices,answerKey,options,prompt
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...


In [24]:
len(arc_df), len(arc_df['id'].unique())

(1119, 1119)

In [129]:
PROCESSED_DATA_PATH = "./datasets/arc_challenge_train_gpt4_response.json"

In [130]:
processed_data = {}

In [131]:
if os.path.exists(PROCESSED_DATA_PATH):
    with open(PROCESSED_DATA_PATH, 'r') as file:
        processed_data = json.load(file)

In [132]:
len(processed_data)

500

In [133]:
def process_row(row):
    id = row['id']
    if id not in processed_data:
        prompt = row['prompt']
        text_response, prompt_tokens, completion_tokens, total_tokens = get_chat_completion(prompt=prompt, model="gpt-4-0125-preview")
        processed_data[id] = {
            'response': text_response,
            'prompt_tokens': prompt_tokens,
            'completion_tokens': completion_tokens,
            'total_tokens': total_tokens,
            'correct_answer': row['answerKey']
        }
        return True
    return False

In [134]:
processed_status = arc_df.iloc[500:1000].progress_apply(lambda row: process_row(row), axis=1)

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [2:48:47<00:00, 20.26s/it]  


In [135]:
len(processed_data)

1000

In [136]:
with open(PROCESSED_DATA_PATH, 'w') as file:
	json.dump(processed_data, file)

In [137]:
len(processed_status[processed_status])

500